In [ ]:
!pip install tensorflow==1.13.1 ;
!pip install keras==2.2.4
import tensorflow as tf
print(tf.__version__)
!pip install papermill

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Global settings and imports

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/newsrec model/Recommenders")
from reco_utils.recommender.deeprec.D2NNutils.py import download_deeprec_resources 
from reco_utils.recommender.newsrec.Newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.D2NN_simple import D2NNModel
from reco_utils.recommender.newsrec.io.D2NN_iterator import D2NNIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf
import os

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

## Download and load data

In [ ]:
data_path = '/content/drive/My Drive/newsrec model/datasets/D2NN'
yaml_file = os.path.join(data_path, r'D2NN.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')


## Create hyper-parameters

In [ ]:
epochs=5
seed=42

In [ ]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epochs)
print(hparams)

In [ ]:
iterator = D2NNIterator

## Train D2NN model

In [ ]:
model = D2NNModel(hparams, iterator, seed=seed)

In [ ]:
print(model.run_eval(valid_file))

{'group_auc': 0.5297, 'mean_mrr': 0.1744, 'rmse': 0.7068, 'acc_score': 0.8414, 'f1score': 0.1252, 'recal_score': 1.0, 'prec_score': 0.0707, 'ndcg@5': 0.1741, 'ndcg@10': 0.2377, 'ndcg@20': 0.2978, 'ndcg@50': 0.3487, 'ndcg@100': 0.3701, 'prec@5': 0.9936, 'prec@10': 0.946, 'prec@20': 0.8561, 'prec@50': 0.6343, 'prec@100': 0.4131, 'recal@5': 0.0187, 'recal@10': 0.0373, 'recal@20': 0.0746, 'recal@50': 0.1866, 'recal@100': 0.3731, 'div@5': 0.6759, 'div@10': 0.8234, 'div@20': 0.8934, 'div@50': 0.9244, 'div@100': 0.9289, 'ap_score': 0.0707}


In [ ]:
model.fit(train_file, valid_file)

In [ ]:
res_syn = model.run_eval(valid_file)
print(res_syn)
pm.record("res_syn", res_syn)

## Save the model

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "d2nn_ckpt"))

## Output Prediction File

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file)
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')


f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()